# Opis
Ten notes zawiera funkcje do łączenia i eksplorowania danych. Najpierw dane łączone są w jeden graf wiedzy, a następnie wykonywane są na nich różne zapytania.
## Opisy poszczególnych funkcji

- create_graph(directory_path_with_rdfs)

    Tworzy graf z danymi z plików `.rdf`.

    Wymagane parametry:
    - **directory_path_with_rdfs** - ścieżka do folderu z plikami w formacie `.rdf`

- visualise_graph(graph):

    Tworzy plik html z wizualizacją grafu.

    Wymagane parametry:
    - **graph** - graph stworzony przez `create_graph()`

In [13]:
from rdflib import Graph, Namespace
import glob

In [2]:
def create_graph(directory_path_with_rdfs, generate_files):
    graph = Graph()

    for rdf_file in glob.glob(directory_path_with_rdfs):
        graph.parse(rdf_file)
    print(f"Łącznie wczytano {len(graph)} trójek.")

    if generate_files:
        graph.serialize("./data/merged_graph.rdf", format="xml")
        graph.serialize("./data/merged_graph_turtle.ttl", format="turtle")

    return graph

In [3]:
rdfs_directory_path = "./data/part1/*rdf"
g = create_graph(rdfs_directory_path, False)

Łącznie wczytano 3388 trójek.


In [4]:
# właściwości używane w grafie

q = """
SELECT DISTINCT ?p WHERE {
    ?s ?p ?o .
}
ORDER BY ?p
"""
for row in g.query(q):
    print(row[0])


http://purl.org/dc/elements/1.1/coverage
http://purl.org/dc/elements/1.1/creator
http://purl.org/dc/elements/1.1/date
http://purl.org/dc/elements/1.1/description
http://purl.org/dc/elements/1.1/format
http://purl.org/dc/elements/1.1/identifier
http://purl.org/dc/elements/1.1/issued
http://purl.org/dc/elements/1.1/language
http://purl.org/dc/elements/1.1/publisher
http://purl.org/dc/elements/1.1/relation
http://purl.org/dc/elements/1.1/rights
http://purl.org/dc/elements/1.1/source
http://purl.org/dc/elements/1.1/subject
http://purl.org/dc/elements/1.1/title
http://purl.org/dc/elements/1.1/type


In [15]:
# sprawdzenie wartości dla kilku właściwości

fields = ["dc:coverage", "dc:type", "dc:creator", "dc:format", "dc:publisher", "dc:relation", "dc:subject"]

dc = Namespace("http://purl.org/dc/elements/1.1/")

for field in fields:
    q = f"""
    SELECT DISTINCT ?value WHERE {{
        ?s {field} ?value .
    }}
    LIMIT 20
    """
    print(10*"*"+field+10*"*")
    for row in g.query(q, initNs={"dc": dc}):
        print(row[0])
    print()


**********dc:coverage**********
20 w.
19 w.
19-20 w.
20-21 w.

**********dc:type**********
rozprawa doktorska
druk ulotny
afisz
plakat
klepsydra

**********dc:creator**********
Gęsiak, Leszek
Labour Party
Cambridge Anglo-Polish Society

**********dc:format**********
image/x.djvu
image/x.djvu
application/octet-stream
application/octet-stream
pdf

**********dc:publisher**********
Uniwersytet Jagielloński w Krakowie
I. Związkowa Drukarnia
Jermułowicz i Bergman
M. Stochalski

**********dc:relation**********
oai:jbc.bj.uj.edu.pl:publication:180039
oai:jbc.bj.uj.edu.pl:publication:180039
oai:jbc.bj.uj.edu.pl:publication:421001
oai:jbc.bj.uj.edu.pl:publication:421001
oai:jbc.bj.uj.edu.pl:publication:535690
oai:jbc.bj.uj.edu.pl:publication:535691
oai:jbc.bj.uj.edu.pl:publication:579328
oai:jbc.bj.uj.edu.pl:publication:579397
oai:jbc.bj.uj.edu.pl:publication:579399
oai:jbc.bj.uj.edu.pl:publication:579419
oai:jbc.bj.uj.edu.pl:publication:599804
oai:jbc.bj.uj.edu.pl:publication:599842
oai:jbc.bj.

In [ ]:
# 5 przykładowych dokumentów

sample = """
SELECT DISTINCT ?s WHERE {
    ?s dc:title ?t .
}
LIMIT 5
"""

docs = [row[0] for row in g.query(sample, initNs={"dc": dc})]

for doc in docs:
    print("--- DOCUMENT ---")
    q = f"""
    SELECT ?p ?o WHERE {{
        <{doc}> ?p ?o .
    }}
    """
    for row in g.query(q):
        print(row[0], "→", row[1])
    print()


--- DOCUMENT ---
http://purl.org/dc/elements/1.1/publisher → Uniwersytet Jagielloński w Krakowie
http://purl.org/dc/elements/1.1/subject → religion
http://purl.org/dc/elements/1.1/subject → society
http://purl.org/dc/elements/1.1/subject → wartości rdzenne
http://purl.org/dc/elements/1.1/subject → state
http://purl.org/dc/elements/1.1/subject → państwo
http://purl.org/dc/elements/1.1/subject → core values
http://purl.org/dc/elements/1.1/subject → wielokulturowość
http://purl.org/dc/elements/1.1/subject → multiculturalism
http://purl.org/dc/elements/1.1/subject → społeczeństwo
http://purl.org/dc/elements/1.1/subject → religia
http://purl.org/dc/elements/1.1/relation → oai:jbc.bj.uj.edu.pl:publication:180039
http://purl.org/dc/elements/1.1/relation → oai:jbc.bj.uj.edu.pl:publication:180039
http://purl.org/dc/elements/1.1/format → image/x.djvu
http://purl.org/dc/elements/1.1/format → image/x.djvu
http://purl.org/dc/elements/1.1/creator → Gęsiak, Leszek
http://purl.org/dc/elements/1.1/sour

## Wnioski

Lista wszytskich właściwości: title, creator, date, issued, description, coverage, subject, type, format, identifier, publisher, relation, language, source, rights

**Co na pewno przyda się: title, date, description, subject, identifier**

Co może się przyda: creator, issued, type, publisher, language

Co się NIE przyda: coverage, format, relation, source, rights

In [18]:
# sprawdzenie wartości dla niepewnych właściwości

fields = ["dc:creator", "dc:issued", "dc:type", "dc:publisher", "dc:language"]

dc = Namespace("http://purl.org/dc/elements/1.1/")

for field in fields:
    q = f"""
    SELECT DISTINCT ?value WHERE {{
        ?s {field} ?value .
    }}
    LIMIT 20
    """
    print(10*"*"+field+10*"*")
    for row in g.query(q, initNs={"dc": dc}):
        print(row[0])
    print()

**********dc:creator**********
Gęsiak, Leszek
Labour Party
Cambridge Anglo-Polish Society

**********dc:issued**********
2004
ca 1946
1948
ca 1948
ca 1900
1893
ca 1882
1879
1860
1859
1872
1874
1884
[1930-1936]
[1878]
1944
[1901]
[1896]
1966
1966

**********dc:type**********
rozprawa doktorska
druk ulotny
afisz
plakat
klepsydra

**********dc:publisher**********
Uniwersytet Jagielloński w Krakowie
I. Związkowa Drukarnia
Jermułowicz i Bergman
M. Stochalski

**********dc:language**********
pol
eng
ger
yid
fre
rus



In [6]:
# NIE DZIAŁA DLA DUŻEJ LICZBY DANYCH - GRAF JEST NIECZYTELNY

from pyvis.network import Network

def visualise_graph(graph):
    net = Network(notebook=False, directed=True)
    net.toggle_physics(False)

    for s, p, o in graph:
        net.add_node(str(s), title=str(s))
        net.add_node(str(o), title=str(o))
        net.add_edge(str(s), str(o), title=str(p))

    net.write_html("graf_jbc.html")

#visualise_graph(g)